In [ ]:
import sys
sys.path.append("/nydl/code/open-metric-learning/")

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import yaml
from pathlib import Path
from IPython.core.display import HTML
from IPython.display import display
import torch
import random

from oml.lightning.entrypoints.validate import pl_val
from oml.lightning.callbacks.metric import MetricValCallback
from oml.const import MOCK_DATASET_PATH
from oml.utils.download_mock_dataset import download_mock_dataset

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

%matplotlib inline

In [ ]:
cfg = f"""
    dataset_root: /nydl/data/DeepFashion_InShop/
    dataframe_name: df.csv
    logs_root: logs
    bs_val: 128
    num_workers: 10
    
    transforms_val:
      name: norm_resize_hypvit_torch
      args:
        im_size: 224
        crop_size: 224

    model:
      name: vit
      args:
        arch: vits16
        normalise_features: False
        use_multi_scale: False
        weights: vits16_inshop

    metric_args:
      cmc_top_k: [1, 5, 10]
      map_top_k: [5]
      
      
    accelerator: gpu
    devices: 1
  
"""

In [ ]:
trainer, ret_dict = pl_val(yaml.load(cfg, Loader=yaml.Loader));
clb_metric = [x for x in trainer.callbacks if isinstance(x, MetricValCallback)][0]

In [ ]:
cfg_p =  cfg + f"""
    postprocessor:
      name: pairwise_images
      args:
        top_n: 5
        pairwise_model:
          name: concat_siamese
          args:
            mlp_hidden_dims: [384, 192]
            weights: /nydl/logs/DeepFashion_InShop/2023-02-01_15-57-15_postprocessing/checkpoints/best.ckpt
            extractor:
              name: vit
              args:
                arch: vits16
                normalise_features: False
                use_multi_scale: False
                weights: null
        transforms:
          name: norm_resize_hypvit_torch
          args:
            im_size: 224
            crop_size: 224
        num_workers: 10
        batch_size: 128
        verbose: True

"""

In [ ]:
trainer_p, ret_dict_p = pl_val(yaml.load(cfg_p, Loader=yaml.Loader));
clb_metric_p = [x for x in trainer_p.callbacks if isinstance(x, MetricValCallback)][0]

In [ ]:
cmc_1 = clb_metric.metric.metrics_unreduced["OVERALL"]["cmc"][1]
cmc_1_p = clb_metric_p.metric.metrics_unreduced["OVERALL"]["cmc"][1]

In [ ]:
ids = random.sample(torch.nonzero(cmc_1_p > cmc_1).squeeze().tolist(), 10)

for idx in ids:
    fig = clb_metric.metric.get_plot_for_queries([idx], n_instances=4, verbose=False)
    fig = clb_metric_p.metric.get_plot_for_queries([idx], n_instances=4, verbose=False)
